In [4]:
# import libraries

import os
import os.path
os.chdir('C:\\...')

import urllib.request
import zipfile

import glob

import math
from itertools import combinations

import pandas as pd
import numpy as np
from numpy import random as rd

import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import matplotlib.animation

import networkx as nx
import community

import time
from datetime import datetime
from datetime import timedelta

from pytrends.request import TrendReq

# Ouverture des fichiers

In [2]:
# write on data.csv the name of datas to save on the computer

def filtrer(src, dst):
    date = ['2017120','20171210','20171211']
    
    for line in src:
        if 'mentions.CSV' not in line:
            continue
        arr = line.split(' ')
        for time in date:
            if time in arr[2]:
                dst.write(arr[0]+' '+arr[1]+' '+arr[2])
    
# Ouverture du fichier source
source = open("master_data_en.txt", "r")
 
# Ouverture du fichier destination
destination = open("data.csv", "w")
 
# Appeler la fonction de traitement
filtrer(source, destination)
 
# Fermeture du fichier destination
destination.close()
 
# Fermerture du fichier source
source.close()

FileNotFoundError: [Errno 2] No such file or directory: 'master_data_en.txt'

In [3]:
# download and dezip files from data.csv

def dezip(filezip, pathdst = ''): 
    if pathdst == '': pathdst = os.getcwd()  ## on dezippe dans le repertoire locale 
    zfile = zipfile.ZipFile(filezip, 'r') 
    for i in zfile.namelist():  ## On parcourt l'ensemble des fichiers de l'archive 
        print (i) 
        if os.path.isdir(i):   ## S'il s'agit d'un repertoire, on se contente de creer le dossier 
            try: os.makedirs(pathdst + os.sep + i) 
            except: pass 
        else: 
            try: os.makedirs(pathdst + os.sep + os.path.dirname(i)) 
            except: pass 
            data = zfile.read(i)                   ## lecture du fichier compresse 
            fp = open(pathdst + os.sep + i, "wb")  ## creation en local du nouveau fichier 
            fp.write(data)                         ## ajout des donnees du fichier compresse dans le fichier local 
            fp.close() 
    zfile.close() 

#enregistrement fichier csv
master_data = pd.read_csv('data.csv',sep= ' ',header = None,engine='python')
master_data.columns = ['1','2','url']

#suppression lignes inutiles
for index, row in master_data.iterrows():
    if '.mentions.CSV' not in row['url']:
        master_data = master_data.drop(index)

# vide le dossier où stocker les donnees
for element in os.listdir('C:\\...'):
    os.remove(element)
        
#telecharge et dezippe les fichiers depuis le web
for index, row in master_data.iterrows():
    element = row['url']
    urllib.request.urlretrieve(element,'file')
    dezip('file', 'data_folder')

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
# # create the dataframe from all files of the GDELT dataset

# # all datas have to be in the same folder
# all_data= pd.DataFrame({'ID': [], 'EventDate': [], 'MentionDate' : [], 'Source': [], 
#                         'Article': [], 'Sentence': []}, 
#                        columns = ['ID', 'EventDate', 'MentionDate', 'Source', 
#                                   'Article', 'Sentence'])

# # choose the number of files to use
# # one file per quarter hour : one week = 692
# max_iter = 692
# compte = 0

# for element in os.listdir('C:\\...):
#     if '.CSV' in element:
#         if compte < max_iter:
#         #enregistrement fichier csv
#             new_data = pd.read_csv('wikileaks\\'+element,parse_dates=[1,2],sep= '\t', decimal= '.',header = None)
#             #suppression colonnes inutiles
#             for c in new_data.columns:
#                 if c not in [0,1,2,4,5,6]:
#                     del new_data[c]
#             new_data.columns = ['ID','EventDate','MentionDate','Source','Article','Sentence']
#             all_data = pd.concat([all_data,new_data])
#             print(element+' : done')
#             compte +=1
# print("completed")

# # save the dataframe into a csv file to use it next time

# all_data.to_csv('all_data.csv', sep= '\t', decimal= '.')

# # read the csv file and put it in a dataframe
# all_data = pd.read_csv('all_data.csv',parse_dates=[1,2],sep = '\t',decimal = '.')
# for c in all_data.columns:
#     if c not in  ['ID', 'EventDate', 'MentionDate', 'Source', 'Article', 'Sentence']:
#         del all_data[c]

In [ ]:
# number of events, articles and mention_date of the dataset
print('#events :', len(all_data.ID.unique()), '#sources :', len(all_data.Source.unique()), '#articles', len(all_data.Article.unique()), '#mention_date', len(all_data.MentionDate.unique()))

In [ ]:
# source graphe
# nodes : sources / edges : 2 sources shares the same event in the same mention date
# edges weight : number of shared events
# /!\ some sources are not represented in the graphe if they are not linked to a least one source

G = nx.Graph()

for date in all_data.MentionDate.unique():
    data_t = all_data[all_data['MentionDate'].isin([date])]
    for event in data_t.ID.unique():
        data_e = data_t[data_t['ID'].isin([event])]
        for comb in combinations(data_e['Source'],2):
            if comb[0]!=comb[1]:
                if G.has_edge(comb[0],comb[1]):
                    G[comb[0]][comb[1]]['weight'] += 1
                else:
                    G.add_edge(comb[0], comb[1], weight=1)
                    
nx.write_gexf(G,'time_source_1.gexf')

In [ ]:
# source graphe
# nodes : sources / edges : 2 sources shares the same event
# edges weight : number of shared events
# /!\ take to much time for 1 week dataset

G = nx.Graph()

for event in all_data.ID.unique():
    data_e = all_data[all_data['ID'].isin([event])]
    for comb in combinations(data_e['Source'],2):
        if comb[0]!=comb[1]:
            if G.has_edge(comb[0],comb[1]):
                G[comb[0]][comb[1]]['weight'] += 1
            else:
                G.add_edge(comb[0], comb[1], weight=1)
nx.write_gexf(G,'time_source_2.gexf')

In [ ]:
# remove edges with a weight < 100

edges_to_remove = []

H = G.copy()

for edge in H.edges():
    if H[edge[0]][edge[1]]['weight']<100:
        edges_to_remove.append(edge)

H.remove_edges_from(edges_to_remove)

nx.write_gexf(H,'connected_source_filtered.gexf')

In [ ]:
# graphe des sources à partir du fichier graphml

# lit le fichier graphml comportant le graphe souhaité
H = nx.read_graphml(path='sources.graphml')

# détection de communauté
partition = community.best_partition(H)

In [ ]:
# Positions des sources à partir de fichier graphml de gephi
pos = {}

# Ouverture du fichier source 
source = open("sources.graphml", "r")

# Appeler la fonction de traitement
for line in source:
    if '<node id' in line:
        line_i = line.replace('<node id="',"")
        node = line_i.replace('">\n',"")
    if '<data key="x"' in line:
        line_i = line.replace('<data key="x">',"")
        x = line_i.replace('</data>\n',"")
        pos[node] = [float(x)]
    if '<data key="y"' in line:
        line_i = line.replace('<data key="y">',"")
        y = line_i.replace('</data>\n',"")
        pos[node].append(float(y))

# Fermerture du fichier source
source.close()

In [ ]:
# fonction de Lissage d'une courbe par moyenne glissante. Le paramètre p determine la qualité du lissage
def lissage(Lx,Ly,p):
    Lxout=[]
    Lyout=[]
    Lxout = Lx[p: -p]
    for index in range(p, len(Ly)-p):
        average = sum(Ly[index-p : index+p+1]) / (2*p + 1)
        Lyout.append(average)
    return Lxout,Lyout

In [ ]:
# article_dict = {source : #articles}
# /!\ 45 MINUTES D'ATTENTE /!\
article_dict = {}
for src in all_data.Source.unique():
    article_dict[src] = len(all_data[all_data['Source'].isin([src])].Article.unique())
    
article_dict_sorted = sorted(article_dict.items(), key=lambda t: t[1])

In [ ]:
# plot the 10 most recurrent sources of the dataset

rec_sources = article_dict_sorted[-11:]

plt.style.use(['fivethirtyeight','seaborn-white'])

source = []
frequency = []

for i in range(len(rec_sources)):
    source.append(rec_sources[i][0])
    frequency.append(rec_sources[i][1])

plt.figure(1)
plt.bar(source, frequency)
plt.xticks(rotation='vertical')
plt.ylabel('#Articles')
plt.title('Most recurrent sources')
plt.legend()
plt.tight_layout()
plt.savefig('most_recurrent_sources.png')
plt.show()

In [ ]:
# plot the five heaviest links of the graphe

edge_list = [('wacotrib.com\nchieftain.com', 5515.0), 
            ('wacotrib.com\ngreensboro.com', 5310.0),
            ('greensboro.com\nroanoke.com', 4906.0), 
            ('wacotrib.com\nroanoke.com', 4706.0), 
            ('greensboro.com\nchieftain.com', 4705.0)]

fig = figure(2, figsize=(10, 6))
plt.style.use(['fivethirtyeight','seaborn-white'])

source = []
frequency = []

for i in range(len(edge_list)):
    source.append(edge_list[i][0])
    frequency.append(edge_list[i][1])

plt.figure(2)
plt.bar(source, frequency)
plt.ylabel('Weight')
plt.title('Most connected sources')
plt.legend()
plt.savefig('most_connected_sources.png')
plt.show()

In [ ]:
# taux de republication pour les 5 sources les plus influentes du web
# calculé sur 500 events pris aléatoirement pour chaque source

rep_rate = {}
pop_source = ['yahoo.com','cnn.com','nytimes.com','msn.com','breitbart.com']
for source in pop_source:
    rep_rate[source] = [100,0]
    data_source = all_data[all_data['Source'].isin([source])]
    compteur = 0
    event_liste = []
    events = data_source.ID.unique()

    while compteur < 500:
        event = random.choice(events)
        event_liste.append(event)
        compteur+=1
            
    for event in  event_liste:
        source_mention = data_source[data_source['ID'].isin([event])]['MentionDate'].iloc[0]
        first_mention = all_data[all_data['ID'].isin([event])]['MentionDate'].iloc[0]
        if source_mention == first_mention:
            rep_rate[source][1] += 1

rate = [1-rep_rate[k][1]/rep_rate[k][0] for k in rep_rate.keys()]

In [ ]:
# Google Trends by Volodymyr Miz

plt.style.use('seaborn-whitegrid')

keywords = ['wikileaks']

pytrend = TrendReq(hl='en-US')
pytrend.build_payload(kw_list=keywords, timeframe='2017-12-01 2017-12-20', geo='', gprop='')

interest_over_time_df = pytrend.interest_over_time()
interest_over_time_df.shape
interest_over_time_df.plot()
plt.savefig('google_trends.png')
plt.show()

In [ ]:
n_heavy = {}
for n in [node for node in partition.keys() if partition[node]==4]:
    neigbors = {k : H[n][k]['weight'] for k in H[n].keys()}
    neigbors_sorted = sorted(neigbors.items(), key=lambda t: t[1])
    n_heavy[n] = neigbors_sorted[-1]

In [ ]:
# heaviest connections in the graph
for edge in H.edges():
    if H[edge[0]][edge[1]]['weight'] > 3000:
        print(edge,partition[edge[0]],partition[edge[1]])

In [ ]:
# informations sur les liaisons

heav_com = {}
for com in range(0,11):
#     max_heav = 0.
#     for n in [node for node in partition.keys() if partition[node]==com]:
#         neigbors = {k : H[n][k]['weight'] for k in H[n].keys()}
#         neigbors_sorted = sorted(neigbors.items(), key=lambda t: t[1])
#         if neigbors_sorted[-1][1] > max_heav:
#             max_heav = neigbors_sorted[-1][1]
#     heav_com[com] = max_heav
    w = 0
    tot = 0
    for n in [node for node in partition.keys() if partition[node]==com]:
        for k in H.neighbors(n):
            if partition[k] == com:
                w += H[n][k]['weight']
                tot += 1
    heav_com[com] = w/tot

In [ ]:
# génère une palette de n couleurs aléatoires

def RGB_to_hex(RGB):
#   [255,255,255] -> "#FFFFFF"
    # Components need to be integers for hex to make sense
    RGB = [int(x) for x in RGB]
    return "#"+"".join(["0{0:x}".format(v) if v < 16 else
            "{0:x}".format(v) for v in RGB])

def rand_hex_color(num=1):
#     Generate random hex colors, default is one, returning a string. If num is greater than
#     1, an array of strings is returned.
    colors = [
    RGB_to_hex([x*255 for x in rnd.rand(3)])
    for i in range(num)
    ]
    if num == 1:
        return colors[0]
    else:
        return colors

n = len(list(set(list(partition.values()))))

colors_list = rand_hex_color(n)

In [ ]:
x = []
y = []
n_col = []

fig = figure(1, figsize=(10, 10))

x_min =-3000
x_max =3000
y_min =-3000
y_max =3000

# x_min = 450
# x_max = 550
# y_min = 450
# y_max = 550

for node in pos.keys():
    if pos[node][0]>x_min and pos[node][0]<x_max and pos[node][1]>y_min and pos[node][1]<y_max:
        x.append(pos[node][0])
        y.append(pos[node][1])
        col = colors_list[partition[node]]
        n_col.append(col)
plt.figure(1)
plt.scatter(x, y, s=1, color = n_col)
plt.title('Linked source')
plt.savefig('linked_source_communities_1.png')
plt.show()

In [ ]:
# affiche les communautés présent dans le cadre [(x_min, x_max), (y_min, y_max)]

x_min = 450
x_max = 550
y_min = 450
y_max = 550

for node in pos.keys():
    if pos[node][0]>x_min and pos[node][0]<x_max and pos[node][1]>y_min and pos[node][1]<y_max:
        print(partition[node])

In [ ]:
# affiche les sources appartenant à la communauté n et ayant écrit plus de 100 articles sur la période
# affiche quelques statistiques sur la comnunauté n

communauté = [node for node in partition.keys() if partition[node] == n]
art = []
n =

for source in communauté:
    if len(all_data[all_data['Source'].isin([source])].Article.unique())>100:
        print(source)

for source in communauté:
    art.append(len(all_data[all_data['Source'].isin([source])].Article.unique()))
    
art_np = np.array(art)

art_m = np.mean(art_np, 0)
art_std = np.std(art_np, 0, ddof=1)
art_M = np.median(art_np, 0)
print('mean : %f, std : %f, med : %f, tot : %d' %(art_m,art_std,art_M,len(communauté)))

In [ ]:
source_pos_df= pd.DataFrame({'Position_x': x, 'Position_y': y}, 
                            columns = ['Position_x', 'Position_y'])

In [ ]:
# Custom the color
sns.set(style="white", color_codes=True)

# Custom the inside plot: options are: “scatter” | “reg” | “resid” | “kde” | “hex”
sns_plot = sns.jointplot(x = source_pos_df['Position_x'], y = source_pos_df['Position_y'], kind='kde', color="skyblue")
sns_plot.savefig('kde_source.png')

In [ ]:
compteur=0
for index,row in data_event.iterrows():
    if compteur == 10:
        continue
    print(row['Article'],row['Sentence'])
    compteur+=1

In [ ]:
source_on = {}
for key in dict_mention.keys():
    if key in pos:
        if dict_mention[key] in source_on:
            source_on[dict_mention[key]].append(pos[key])
        else:
            source_on[dict_mention[key]] = [pos[key]]

In [ ]:
frames = int(max(key for key in source_on.keys()))
print(frames)

In [ ]:
x = []
y = []
for event in event_liste:
    data_event = all_data[all_data['ID'].isin([event])]
    quantity = len(data_event)
    ti = data_event.iloc[0]['MentionDate']
    tf = data_event.iloc[-1]['MentionDate']
    ti_dt = datetime.strptime(ti, '%Y-%m-%d %X')
    tf_dt = datetime.strptime(tf, '%Y-%m-%d %X')
    delta_t = (tf_dt - ti_dt).seconds
    delta_t /= 60
    x.append(int(delta_t))
    y.append(quantity)
    
time_qty_df= pd.DataFrame({'Qty': y, 'Time': x}, columns = ['Qty', 'Time'])

# Custom the color
sns.set(style="white", color_codes=True)

# Custom the inside plot
sns_plot = sns.jointplot(x = time_qty_df['Time'], y = time_qty_df['Qty'], kind='kde', color="skyblue")
sns_plot.savefig('time_vs_qty_kde.png')

time_qty_df_z = time_qty_df.query('Qty < 150')

# Custom the inside plot
sns_plot = sns.jointplot(x = time_qty_df_z['Time'], y = time_qty_df_z['Qty'], kind='kde', color="skyblue")
sns_plot.savefig('time_vs_qty_z_kde.png')


In [ ]:
# zoom sur le noyau central

x_z = []
y_z = []
pos_z = {}
        
for source in pos.keys():
    if abs(pos[source][0]) < 3000 and abs(pos[source][1]) < 3000:
        x_z.append(pos[source][0])
        y_z.append(pos[source][1])
        pos_z[source] = pos[source]
        
plt.plot(x_z,y_z,ms=0.3,color='k',marker='o',ls='')

plt.title('Linked source')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# dictionnaire dict_mention = {source : delta_t}

dict_mention = {}

# data_event = all_data[all_data['ID'].isin([713561281.0])]
data_event = all_data[all_data['ID'].isin([714037633.0])]

t0 = data_event.iloc[0]['MentionDate']
for index, row in data_event.iterrows():
    if row['Source'] in dict_mention:
        continue 
    t = row['MentionDate']
    delta_t = (t - t0).seconds
    delta_t /= 60
    dict_mention[row['Source']] = int(delta_t)

In [ ]:
# event source dictionnaire 

source_on = {}
for key in dict_mention.keys():
    if key in pos_z:
        if dict_mention[key] in source_on:
            source_on[dict_mention[key]].append(pos[key])
        else:
            source_on[dict_mention[key]] = [pos[key]]

In [ ]:
frames = int(max(key for key in source_on.keys())/15)
print(frames)

In [ ]:
# animate the propagation of an event

os.system('ffmpeg')

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(-3000, 3000), ylim=(-3000, 3000))
scat, = ax.plot([], [], ms=0.2,color='k',marker='o',ls='')
new_scat, = ax.plot([], [], ms=1,color='red',marker='o',ls='')
u = []
v = []

# initialization function: plot the background of each frame
def init():
    for p in pos_z.values():
        x.append(p[0])
        y.append(p[1])
    scat.set_data(x, y)
    return scat,

# animation function.  This is called sequentially
def animate(time):
    if time*5 in source_on:
        for point in source_on[time*5]:
            u.append(point[0])
            v.append(point[1])
    new_scat.set_data(u, v)
    return new_scat,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=frames, blit=True)

# save the animation as an mp4 where 1 second is equivalent to 1 hour
anim.save('714037633_real.mp4', fps=12, extra_args=['-vcodec', 'libx264'])

plt.show()

In [ ]:
# zoom sur le noyau central

x_z = []
y_z = []
pos_z = {}
        
for source in pos.keys():
    if pos[source][0] < -500 and pos[source][0] > -1250 and pos[source][1] < 500 and pos[source][1] > 0:
#     if pos[source][0] < 0 and pos[source][0] > -2000 and pos[source][1] < 1000 and pos[source][1] > -1000:
        x_z.append(pos[source][0])
        y_z.append(pos[source][1])
        pos_z[source] = pos[source]
        print(source)
        
plt.plot(x_z,y_z,ms=2,color='k',marker='o',ls='')

plt.title('Linked source')
plt.xlabel('x')
plt.ylabel('y')
plt.savefig('wiki_source_z.png')
plt.show()

In [ ]:
event_list = all_data.ID.unique()
compteur = 0
event = random.choice(event_list)

while len(all_data[all_data['ID'].isin([event])]) < 130 or compteur < 3:
    event = random.choice(event_list)
    if len(all_data[all_data['ID'].isin([event])]) >= 130:
        compteur +=1
        print(compteur, event)

In [ ]:
#fonction de Lissage d'une courbe par moyenne glissante. Le paramètre p determine la qualité du lissage
def lissage(Lx,Ly,p):
    Lxout=[]
    Lyout=[]
    Lxout = Lx[p: -p]
    for index in range(p, len(Ly)-p):
        average = sum(Ly[index-p : index+p+1]) / (2*p + 1)
        Lyout.append(average)
    return Lxout,Lyout

In [ ]:
# How to find fake news?

# fake_news return sorted event dictionnary mentionned by news_src with the number of mentions in other sources
def news(news_src):
    c=0 # impose une limite si il y a trop d'event pour une source donnée
    news_dict = {}
    news_data = all_data[all_data['Source'].isin([news_src])]
    for id in news_data.ID.unique():
        if all_data[all_data['ID'].isin([id])]['MentionDate'].iloc[0] == news_data[news_data['ID'].isin([id])]['MentionDate'].iloc[0]:
            news_dict[id] = len(all_data[all_data['ID'].isin([id])])
        if c == 1000:
            break
        c+=1

    news_dict_sorted = sorted(news_dict.items(), key=lambda t: t[1])
    
    return(news_dict_sorted, news_dict)

# checking sources bias : mediabiasfactcheck.com
# interesting sources : snopes.com, infowars.com, theonion.com, breitbart.com
# InfoWars is a far-right American conspiracy theorist and fake news website and media platform
# The Onion is an American digital media company and news satire organization that publishes articles on 
# international, national, and local news : not enough propagation
# Breitbart News Network is a far-righ syndicated American news, opinion and commentary website

source = 'theonion.com'
D_s, D = news(source)

D_s

In [ ]:
# Find 3 events created by a source which produce more than 100 articles

data_source = all_data[all_data['Source'].isin([source])]
compteur = 0
event_liste = []
events = [k for k in D]

# while compteur < 3:
for event in events:
#         event = random.choice(events)
    source_mention = data_source[data_source['ID'].isin([event])]['MentionDate'].iloc[0]
    first_mention = all_data[all_data['ID'].isin([event])]['MentionDate'].iloc[0]
    if source_mention == first_mention and D[event]>5:
        event_liste.append(event)
        compteur+=1
        
print(event_liste)

In [ ]:
# print the 10 events by the source which produce the highest articles number
for id in D_s[-10:]:
    for index, row in all_data[all_data['ID'].isin([id[0]])].iterrows():
        if row['Source'] == source:
            print(id, row['Article'],row['Sentence'])

In [ ]:
# print some informations about one event
compteur = 0
event = '712374053.0'
for index,row in all_data[all_data['ID'].isin([event])].iterrows():
    print(row['Article'],row['Sentence'])
    compteur += 1
    if compteur == 30:
        break

In [ ]:
# somes news created by some interesting sources

# https://www.infowars.com/u-s-moves-to-criminalize-anonymous-cryptocurrency-ownership/
# https://www.infowars.com/cnns-acosta-when-journalists-attacked-they-must-resist/
infowars_id = ['712370880.0','712369974.0']

# http://www.breitbart.com/jerusalem/2017/12/05/hallelujah-president-donald-trump-will-recognize-jerusalem-as-capitol-of-israel-begin-moving-embassy/ 16.0
# http://www.breitbart.com/big-government/2017/12/05/do-not-talk-to-me-about-honor-and-integrity-steve-bannon-savages-entire-romney-clan-after-mitts-insult-to-vietnam-vet-roy-moore/ 16.0
breitbart_id = ['712726389.0','712766837.0']

# https://www.reuters.com/article/us-honduras-election/honduran-candidate-calls-on-army-to-rebel-amid-disputed-vote-count-idUSKBN1DX0X7 6.0
# https://www.reuters.com/article/us-northkorea-missiles-drill/south-korea-u-s-kick-off-largest-air-exercise-amid-north-korean-warnings-idUSKBN1DY02I 11.0
reuters_id = ['712080245.0', '712090711.0']

# /!\ not enough propagation in other sources : only 14 articles
# https://www.buzzfeed.com/hayesbrown/perry-of-arabia
buzzfeed_id = ['712726898.0']

# /!\ not enough propagation in other sources : only 6 articles
# https://politics.theonion.com/melania-trump-hangs-decayed-badger-carcass-over-white-h-1820886857
theonion_id = ['712374053.0']

# http://www.zerohedge.com/news/2017-12-05/erdogan-tells-trump-declaring-jerusalem-capital-israel-would-be-red-line 15.0
# http://www.zerohedge.com/news/2017-12-03/legal-system-failing-america-when-it-comes-immigration 10.0
zerohedge_id = ['712525070.0','712111700.0']

event_dict = {'breitbart' : breitbart_id, 'infowars' : infowars_id, 'reuters' : reuters_id,
              'buzzfeed' : buzzfeed_id, 'zerohedge' : zerohedge_id}

In [ ]:
# liste de dictionnaires dict_mention_list = [{source : delta_t}]
# liste de dictionnaires dict_delta_t_list = [{delta_t : nombre d'article}]
# dictionnaire des events dict_event = [{event : url}]

# 4 colors
four_colors = ['#EC4A94','#F3A530','#88C542','#367ABD']

# 5 colors
five_colors = ['#EE4035','#F0A32F','#4CB2D4','#844D9E','#56B949']

plt.style.use('seaborn-whitegrid')

fig = figure(1, figsize=(24, 8))

i = 0

# event_dict contient 10 real events et 10 fake events
for k in event_dict:
# calcule le nombre d'article par période de temps pour 10 events
    dict_mention_list = []
    dict_delta_t_list = []
    dict_delta_t_cumul_list = []
    dict_event = []
    delta = 15.0

    for event in event_dict[k]:
        D={}
        E={}
        F={}
        data_event = all_data[all_data['ID'].isin([str(event)])]
        t0 = data_event.iloc[0]['MentionDate']
        t0_dt = datetime.strptime(t0, '%Y-%m-%d %X')
        for index, row in data_event.iterrows():
            t = row['MentionDate']
            t_dt = datetime.strptime(t, '%Y-%m-%d %X')
            delta_t = (t_dt - t0_dt).seconds
            delta_t /= 60           
            D[row['Source']] = delta_t
            if delta_t in E:
                E[delta_t]+=1
            else:
                E[delta_t]=1
            if delta_t in F:
                F[delta_t]+=1
            elif F:
                F[delta_t] = max(F[d] for d in F)+1
            else:
                F[delta_t]=1
        dict_mention_list.append(D)
        dict_delta_t_list.append(E)
        dict_delta_t_cumul_list.append(F)
        dict_event.append({event : data_event.iloc[0]['Article']})
    
# graphe du #new articles
    j = 1
    for D in dict_delta_t_list:
        lists = sorted(D.items()) # sorted by key, return a list of tuples
        t, qty = zip(*lists) # unpack a list of pairs into two tuples
        plt.subplot(121)  
        if j == len(event_dict[k]):    
            plt.plot(t, qty,label = k,color=five_colors[i])
        else:
            plt.plot(t, qty, color=five_colors[i])
        j+=1
    plt.xlabel('Time[min]')
    plt.ylabel('#New articles')
    plt.title("#New articles vs Time")
    plt.legend()

# graphe du #articles
    j = 1
    for D in dict_delta_t_cumul_list:
        lists = sorted(D.items()) # sorted by key, return a list of tuples
        t, qty = zip(*lists) # unpack a list of pairs into two tuples
        for key in dict_event[dict_delta_t_cumul_list.index(D)]:
            l = key
        plt.subplot(122)  
        if j == len(event_dict[k]):    
            plt.plot(t,qty,label = k,color=five_colors[i])
        else:
            plt.plot(t,qty,color=five_colors[i])
        j+=1
    plt.xlabel('Times[min]')
    plt.ylabel('#Articles')
    plt.title("#Articles vs Time")
    plt.legend()
    
    plt.savefig('news_across_the_time.png')
    
    i+=1